### Configurações Pyspark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window

import pandas as pd

def write_to_postgresl(df, tb_name=None, write_mode='None'):
    df.count()
    if tb_name is None:
        raise Exception('Informe o nome da tabela')
    if write_mode is None:
        raise Exception('Informe o mode de escrita: append ou overwrite')
        
    from datetime import datetime
    start_time = datetime.now()
    
    try:
        existing_df = spark.read.jdbc(url, table=tb_name, properties=properties)
        print('Table exists, appending data.')
        df.write.jdbc(url, table=tb_name, mode=write_mode, properties=properties)
    except Exception as e:
        print(f'Erro: {e}')
    end_time = datetime.now()
    
    total_time = str(end_time - start_time)
    
    return f'Total time: {total_time} and - Total rows: {df.count()} - Total columns: {len(df.columns)}'



    # Criando a sessão do Spark
spark  = SparkSession.builder \
    .appName("Data Analysis") \
    .config('spark.jars', '/data/IDAF/DATABASECONNECTOR_JAR_FOLDER/postgresql-42.2.18.jar')\
    .config("spark.executor.memory", "8g")\
    .config("spark.driver.memory", "8g")\
    .config("spark.executor.cores", "4")\
    .config("spark.executor.instances","8")\
    .config("spark.sql.shuffle.partitions","96")\
    .config("spark.default.parallelism","96")\
    .getOrCreate()


url = "jdbc:postgresql://localhost:5432/postgres"

properties = {
    "user" : "postgres",
    "password" : "cidacs",
    "driver" : "org.postgresql.Driver"
}

25/07/16 23:17:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/16 23:17:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/07/16 23:17:40 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/07/16 23:17:40 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/07/16 23:17:40 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/07/16 23:17:40 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


### Configurações Pandas

In [2]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100)
spark.conf.set("spark.sql.repl.eagerEval.maxNumRows", 300)

spark.conf.set("spark.sql.debug.maxToStringFields", 100)

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_rows", None)

### Lendo dados enriquecidos

In [3]:
df_input = (spark
            .read
            .parquet('/data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/scripts_omop_tb/basefinal_tb_limp_enriched')
           )

### Location

In [6]:
df_death = (df_input
               .select('person_id_infant', F.col('dtobito_sim'))
               .withColumn('person_id', F.col('person_id_infant'))
            .withColumn('death_date', F.col('dtobito_sim'))
            .filter(F.col('death_date').isNotNull())
            .withColumn('death_datetime', F.lit(None))
            .withColumn('death_type_concept_id', F.lit(32879))
            .withColumn('cause_concept_id', F.lit(None))
            .withColumn('cause_source_value', F.lit(None))
            .withColumn('cause_source_concept_id', F.lit(None))
           ).select(F.col('person_id').cast('integer'),
                    F.col('death_date').cast('date'),
                    F.col('death_datetime').cast('timestamp'),
                    F.col('death_type_concept_id').cast('integer'),
                    F.col('cause_concept_id',).cast('integer'),
                    F.col('cause_source_value').cast('string'),
                    F.col('cause_source_concept_id').cast('integer'))

df_death.count()

215852

### Salvando no Postgresql

In [7]:
write_to_postgresl(df_death, tb_name='tuberculose.death', write_mode='append')

Table exists, appending data.


'Total time: 0:01:09.387705 and - Total rows: 215852 - Total columns: 7'